Importing the required libraries

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


Loading the required x_train, y_train, x_test, y_test

In [3]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv").squeeze()  # Convert to Series
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv").squeeze()

Building the Perceptron

In [7]:
# Convert DataFrame to NumPy array before converting to tensor
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [8]:
# Create DataLoaders
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Training the Perceptron

In [9]:
# Define the SLP model
class SingleLayerPerceptron(nn.Module):
    def __init__(self, input_size):
        super(SingleLayerPerceptron, self).__init__()
        self.fc = nn.Linear(input_size, 1)  # Single layer
        self.sigmoid = nn.Sigmoid()  # Activation function

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

# Initialize the model
input_size = X_train.shape[1]  # Number of features
model = SingleLayerPerceptron(input_size)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        y_batch = y_batch.float().unsqueeze(1)  # Reshape for BCELoss
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/20, Loss: 0.5938
Epoch 2/20, Loss: 0.5097
Epoch 3/20, Loss: 0.4797
Epoch 4/20, Loss: 0.4640
Epoch 5/20, Loss: 0.4548
Epoch 6/20, Loss: 0.4481
Epoch 7/20, Loss: 0.4432
Epoch 8/20, Loss: 0.4402
Epoch 9/20, Loss: 0.4391
Epoch 10/20, Loss: 0.4362
Epoch 11/20, Loss: 0.4356
Epoch 12/20, Loss: 0.4343
Epoch 13/20, Loss: 0.4337
Epoch 14/20, Loss: 0.4337
Epoch 15/20, Loss: 0.4338
Epoch 16/20, Loss: 0.4326
Epoch 17/20, Loss: 0.4319
Epoch 18/20, Loss: 0.4307
Epoch 19/20, Loss: 0.4322
Epoch 20/20, Loss: 0.4315


In [11]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # Convert probabilities to 0 or 1
        correct += (predicted.squeeze() == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7927


In [ ]:
Building a mlp model